In [38]:
#!{sys.executable} -m pip install yfinance
import yfinance as yf
import datetime

In [114]:
#Define stock price predictor class
class stock_price_predictor():
    
    def __init__(self,ticker ):
        import numpy as np
        import pandas as pd
        import plotly.graph_objects as go
        import matplotlib.pyplot as plt
        
        self.stock = ticker
        self.split = 0.8
        #self.app(ticker,days)
        #TO DO:
        #define a default start date, allow users to predict on the basis of data
        self.data = self.app()
        if self.data.shape!=(0,6):
            #print(self.data.shape)
            #print(type(self.data))
            #print(self.data.head())
            pass
            
        else:
            print("koi data nahi hai kaake")

    def app(self):
        
        #Defining default start date and end date
        import datetime
        import yfinance as yf
        start_date = "2000-01-01"
        end_date = datetime.date.today().strftime("%Y-%m-%d")
        print("End Date: ",end_date)
        
        #import libraries for getting stock data
        
        #throws error if the ticker is incorrect
        
        raw_data = yf.Ticker(ticker)
        historical_data = raw_data.history(start = start_date, end = end_date)
        
        #Display some information about the selected ticker
        
        
        #raw_data = yf.Ticker(ticker)
        #historical_data = raw_data.history(start = start_date, end = end_date)
        return historical_data
        
    
    def predict(self,days):
        
        self.shift_days = int(days)
        if self.shift_days not in [1,5,10]:
            print("Incorrect time frame. Currently, the app supports predicting stock price 1/5/10 day(s) in the future")
            return
        
        new_data = self.preprocess(self.data)
        
        #call the price prediction function which in turn calls split_data function
        results = self.predict_price(new_data)
        
        #Define train and test data
        
        #Train the data
        #Predict
        #Share results  
        return results
    
    
    def preprocess(self,data):
        import pandas as pd
        historical_data = data.copy()
        historical_data['Date'] = historical_data.index

        historical_data['Date'] = pd.to_datetime(historical_data['Date'], format = "%Y-%m-%d")
        
        #Change column order
        historical_data = historical_data.iloc[:, [7,0,1,2,3,4,5,6]]
        
        from ta import add_all_ta_features 
        historical_data = add_all_ta_features(historical_data, "Open", "High", "Low", "Close", "Volume", fillna=True)
        
        from fastai.tabular import core

        #Define the date parts
        core.add_datepart(historical_data,'Date', drop = 'True')

        # Ensure the correct format
        historical_data['Date'] = pd.to_datetime(historical_data.index.values, format = "%Y-%m-%d")

        # Add the date parts
        core.add_datepart(historical_data, 'Date', drop = 'True')
        
        
        return historical_data
    

    #Train Test Split the data
    def splitdata(self,df,split,shift):
        #since this is time series data, we pick first 'split'% data for training and the remaining for test
        split_ind = int(len(df)*split)
        

        train_data = df.iloc[:split_ind,:]
        test_data = df.iloc[split_ind:,:]
        #print("Train Data Shape: ",train_data.shape)
        #print("Test Data Shape: ",test_data.shape)

        x_train = train_data.iloc[:-shift,1:-1]
        y_train = train_data['Close_lag'][:-shift]
        x_test = test_data.iloc[:-shift,1:-1]
        y_test = test_data['Close'][:-shift]

        
        return x_train, x_test, y_train,y_test, train_data, test_data
    
    def create_lag(self,df,shift):
        lag_days = -shift
        df['Close_lag'] = df['Close'].shift(lag_days)
        return df



    def predict_price(self, data):

        
        from sklearn.linear_model import LinearRegression
        from sklearn.linear_model import LassoCV
        from sklearn.neural_network import MLPRegressor
        from sklearn.preprocessing import StandardScaler
        from sklearn.metrics import mean_squared_error
        
        df = self.create_lag(data, self.shift_days)

        x_train,x_test,y_train,y_test,train_data,test_data = self.splitdata(df,self.split,self.shift_days)
        #Linear Regression - Linear Regression Model
        lr = LinearRegression()
        lr.fit(x_train,y_train)
        lr_preds = lr.predict(x_test)
        #print(lr_preds.shape)
        lr_mse = mean_squared_error(y_test, lr_preds)
        lr_R2 = lr.score(x_test,y_test)
        #print('Linear Regression R Squared Value: {}'.format(lr_R2))
        print('Linear Regression MSE: {}'.format(round(lr_mse),2))

        #MLP Regressor - Artificial Neural Network

        #********** Scaling data
        scaler = StandardScaler()
        x_train_scaled = scaler.fit_transform(x_train)
        x_test_scaled = scaler.transform(x_test)
        #****************


        MLP = MLPRegressor(random_state=22, max_iter=1000, hidden_layer_sizes = (100,), 
                             shuffle = False, activation = 'identity',
                             learning_rate = 'adaptive').fit(x_train_scaled, y_train)
        MLP.fit(x_train,y_train)
        MLP_preds = MLP.predict(x_test)
        MLP_R2 = MLP.score(x_test, y_test)
        MLP_MSE = mean_squared_error(y_test, MLP_preds)
        #print('Muli-layer Perceptron R2 : {}'.format(MLP_R2))
        print('Multi-layer Perceptron MSE: {}'.format(round(MLP_MSE),2))
        print("Time Period: {}".format(self.shift_days))
        last_trading_date = x_test.index[-1]
        prediction_date = last_trading_date + datetime.timedelta(days=self.shift_days)
        print("Last trading details: {} on {}".format(y_test[-1],x_test.index[-1]))
        print("Prediction for {} : {}".format(prediction_date,lr_preds[-1]))
        print("This is a {}% change.".format(round(100*(lr_preds[-1]-y_test[-1])/y_test[-1],2)))

        #final_preds = (MLP_MSE*lr_preds + MLP_preds*lr_MSE)/(MLP_MSE+lr_mse)

        return lr_preds

In [115]:
#define ticker symbol for price prediction
ticker = "GOOGL"

In [116]:
#class object
predictor = stock_price_predictor(ticker)

End Date:  2022-09-04


In [117]:
#predict price 1/5/10 days ahead
x = predictor.predict(5)

/Users/smoothoperator/opt/anaconda3/envs/smoothenv/lib/python3.7/site-packages/ta/trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/smoothoperator/opt/anaconda3/envs/smoothenv/lib/python3.7/site-packages/ta/trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


Linear Regression MSE: 4.0
Multi-layer Perceptron MSE: 2192953537135914.0
Time Period: 5
Last trading details: 110.33999633789062 on 2022-08-26 00:00:00
Prediction for 2022-08-31 00:00:00 : 111.49184807029087
This is a 1.04% change.
